Using SQLAlchemy to Talk to a Database
=====================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [64]:
#EDIT
from sqlalchemy import *
import datetime
import mediacloud

key = 'YOUR_API_KEY_HERE'
mc = mediacloud.api.MediaCloud(key)



## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [65]:
#EDIT
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:',echo=True) # just save the db in memory for now (ie. not on disk)
metadata = MetaData()
# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('count', Integer),           
    Column('timestamp', DateTime, default=datetime.datetime.now),
)
metadata.create_all(engine) # and create the tables in the database

2016-12-09 14:32:08,116 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-12-09 14:32:08,117 INFO sqlalchemy.engine.base.Engine ()
2016-12-09 14:32:08,118 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-12-09 14:32:08,119 INFO sqlalchemy.engine.base.Engine ()
2016-12-09 14:32:08,120 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("queries")
2016-12-09 14:32:08,120 INFO sqlalchemy.engine.base.Engine ()
2016-12-09 14:32:08,121 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE queries (
	id INTEGER NOT NULL, 
	keywords VARCHAR(400) NOT NULL, 
	count INTEGER, 
	timestamp DATETIME, 
	PRIMARY KEY (id)
)


2016-12-09 14:32:08,122 INFO sqlalchemy.engine.base.Engine ()
2016-12-09 14:32:08,123 INFO sqlalchemy.engine.base.Engine COMMIT


### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [66]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, count, timestamp) VALUES (:id, :keywords, :count, :timestamp)'

In [67]:
#EDIT
labels = ["puppies", "kittens", "Trump", "Clinton"]
for label in labels:
    res = mc.sentenceCount(label, solr_filter=[mc.publish_date_query( datetime.date( 2016, 9, 1), datetime.date( 2016, 10, 1) ), 'tags_id_media:1' ])
    insert_stmt = queries.insert().values(keywords=label, count=res['count'])
    db_conn = engine.connect()
    result = db_conn.execute(insert_stmt)
    result.inserted_primary_key # print out the primary key it was assigned
str(insert_stmt)

2016-12-09 14:32:08,803 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-09 14:32:08,804 INFO sqlalchemy.engine.base.Engine ('puppies', 539, '2016-12-09 14:32:08.802786')
2016-12-09 14:32:08,805 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-09 14:32:09,074 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-09 14:32:09,075 INFO sqlalchemy.engine.base.Engine ('kittens', 350, '2016-12-09 14:32:09.074195')
2016-12-09 14:32:09,076 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-09 14:32:09,618 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) VALUES (?, ?, ?)
2016-12-09 14:32:09,619 INFO sqlalchemy.engine.base.Engine ('Trump', 208498, '2016-12-09 14:32:09.618442')
2016-12-09 14:32:09,621 INFO sqlalchemy.engine.base.Engine COMMIT
2016-12-09 14:32:09,961 INFO sqlalchemy.engine.base.Engine INSERT INTO queries (keywords, count, timestamp) 

'INSERT INTO queries (keywords, count, timestamp) VALUES (:keywords, :count, :timestamp)'

In [68]:
#db_conn = engine.connect()
#result = db_conn.execute(insert_stmt)
#result.inserted_primary_key # print out the primary key it was assigned

In [69]:
#label2 = "kittens"
#res2 = mc.sentenceCount(label2, solr_filter=[mc.publish_date_query( datetime.date( 2016, 9, 1), datetime.date( 2016, 10, 1) ), 'tags_id_media:1' ])
#insert_stmt = queries.insert().values(keywords=label2, count=res2['count'])
#result = db_conn.execute(insert_stmt)
#result.inserted_primary_key # print out the primary key it was assigned

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [70]:
#EDIT
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

2016-12-09 14:32:16,682 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.count, queries.timestamp 
FROM queries
2016-12-09 14:32:16,683 INFO sqlalchemy.engine.base.Engine ()
(1, u'puppies', 539, datetime.datetime(2016, 12, 9, 14, 32, 8, 802786))
(2, u'kittens', 350, datetime.datetime(2016, 12, 9, 14, 32, 9, 74195))
(3, u'Trump', 208498, datetime.datetime(2016, 12, 9, 14, 32, 9, 618442))
(4, u'Clinton', 140533, datetime.datetime(2016, 12, 9, 14, 32, 9, 961169))


In [71]:
#EDIT
select_stmt = select([queries])
count_sum = 0
for row in db_conn.execute(select_stmt):
    count_sum += row[2]
    print row
print "The sum of the sentence counts of the queries is:", count_sum

2016-12-09 14:32:22,061 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.count, queries.timestamp 
FROM queries
2016-12-09 14:32:22,062 INFO sqlalchemy.engine.base.Engine ()
(1, u'puppies', 539, datetime.datetime(2016, 12, 9, 14, 32, 8, 802786))
(2, u'kittens', 350, datetime.datetime(2016, 12, 9, 14, 32, 9, 74195))
(3, u'Trump', 208498, datetime.datetime(2016, 12, 9, 14, 32, 9, 618442))
(4, u'Clinton', 140533, datetime.datetime(2016, 12, 9, 14, 32, 9, 961169))
The sum of the sentence counts of the queries is: 349920


In [9]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

2016-12-09 13:03:10,830 INFO sqlalchemy.engine.base.Engine SELECT queries.id, queries.keywords, queries.timestamp 
FROM queries 
WHERE queries.keywords LIKE ?
2016-12-09 13:03:10,831 INFO sqlalchemy.engine.base.Engine ('p%',)
(1, u'puppies', datetime.datetime(2016, 12, 9, 13, 3, 10, 783332))


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [10]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [11]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    def __repr__(self):
        return "<Query(keywords='%s')>" % (self.keywords)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function now at 0x7f805b00b7d0>)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [12]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [13]:
query = Query(keywords="iguana")
query.keywords

'iguana'

In [14]:
my_session.add(query)
my_session.commit()
query.id

1

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [15]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana')>


In [16]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [17]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana')>
<Query(keywords='robot')>
<Query(keywords='puppy')>


In [18]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q

<Query(keywords='robot')>
